# Architectural deep dive - Llama-Index on AWS

![](./images/llama-index-feed.png)

# Setup

In [ ]:
%pip install -r requirements.txt -qU

In [2]:
# Setting up Bedrock
from llama_index.core import Settings
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding, Models

# Model that will be used to generate the embeddings
Settings.embed_model = BedrockEmbedding(
    model=Models.COHERE_EMBED_MULTILINGUAL_V3, # <---- supports up to 2048 characters per string.
    region_name="us-west-2"
) 
# Model that will be used to generate the answer given the most relevant chunks
Settings.llm = Bedrock(
    model="anthropic.claude-3-haiku-20240307-v1:0", 
    region_name="us-west-2",
    context_size=200000
)

# POC Phase

In [12]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter

# Load Data
documents = SimpleDirectoryReader("data").load_data()
sentence_splitter = SentenceSplitter(chunk_size=250, chunk_overlap=50)
# Create the Vectore Store and Index Store in-memory
index = VectorStoreIndex.from_documents(documents, transformations=[sentence_splitter])

In [ ]:
index.set_index_id("books")
index.storage_context.persist("./storage")

In [ ]:
!ls storage

default__vector_store.json image__vector_store.json
docstore.json              index_store.json
graph_store.json


In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("Chi è Pinocchio?")

In [ ]:
response.response

'Secondo le informazioni fornite nel contesto, Pinocchio è un ragazzaccio, un vagabondo e un vero rompicollo, descritto come disubbidiente e svogliato, che invece di andare a scuola va in giro con i compagni a fare lo sbarazzino. Nonostante il burattino inizialmente cerchi di dipingerlo come un gran buon figliuolo, pieno di voglia di studiare e affezionato alla sua famiglia, alla fine ammette che Pinocchio è davvero un ragazzaccio.'

In [ ]:
for node in response.source_nodes:
    file = node.node.metadata['file_path']
    text = node.node.text
    print(file)
    print(text)

/Users/dggallit/Documents/GenAI/personal-projects-and-experiments/notebooks/llama-index-learning/data/pinocchio.txt
— E chi glielo tirò?

— Un suo compagno di scuola: un certo Pinocchio....

— E chi è questo Pinocchio? — domandò il burattino facendo lo gnorri.

— Dicono che sia un ragazzaccio, un vagabondo, un vero rompicollo.

— Calunnie! Tutte calunnie!

— Lo conosci tu questo Pinocchio?

— Di vista! — rispose il burattino.

— E tu, che concetto ne hai? — gli chiese il vecchietto.

— A me mi pare un gran buon figliuolo, pieno di voglia di studiare,
obbediente, affezionato al suo babbo e alla sua famiglia.... —

Mentre il burattino sfilava a faccia fresca tutte queste bugie, si
toccò il naso e si accorse che il naso gli era allungato più di un
palmo. Allora tutto impaurito cominciò a gridare:

— Non date retta, galantuomo, a tutto il bene che ve ne ho detto;
perchè conosco benissimo Pinocchio e posso assicurarvi anch'io ch'è
davvero un ragazzaccio, un disubbidiente e uno svogliato, e 

# MVP Phase

![](./images/llama-index-feed-mvp.png)

In [ ]:
# Run the docker for chromadb
!docker run -d --name chromadb -p 5000:8000 -v ./chroma:/chroma/chroma -e IS_PERSISTENT=TRUE -e ANONYMIZED_TELEMETRY=TRUE chromadb/chroma:latest
!docker run -d --name redis -p 6379:6379 redis/redis-stack-server:latest

In [ ]:
%pip install llama-index-vector-stores-chroma llama-index-storage-index-store-redis llama-index-storage-docstore-redis -qU

Document stores contain ingested document chunks, which are called `Node` objects.

In [1]:
from llama_index.storage.docstore.redis import RedisDocumentStore

redis_docs = RedisDocumentStore.from_host_and_port(host="0.0.0.0", port=6379, namespace="books-docstore")

Index store contains

In [2]:
# Llama Index basic RAG example
from llama_index.vector_stores.chroma import ChromaVectorStore

chroma = ChromaVectorStore(host="0.0.0.0", port=5000, collection_name="books-vectorstore")

In [3]:
from llama_index.storage.index_store.redis import RedisIndexStore

redis_index = RedisIndexStore.from_host_and_port(host="0.0.0.0", port=6379, namespace="books-indexstore")

In [24]:
from llama_index.core import StorageContext

storage_context = StorageContext.from_defaults(
    docstore=redis_docs, index_store=redis_index, vector_store=chroma)

In [48]:
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter

index = VectorStoreIndex.from_documents(
    documents=SimpleDirectoryReader("data").load_data(), 
    storage_context=storage_context, 
    transformations=[
        SentenceSplitter(chunk_size=448, chunk_overlap=50)
    ]
)
index.set_index_id("books")

In [49]:
query_engine = index.as_query_engine()
query_engine.query("Chi è Pinocchio?")

Response(response=' Sulla base delle informazioni fornite, Pinocchio sembra essere un burattino che inizialmente viene descritto come un "ragazzaccio, un vagabondo, un vero rompicollo" da un vecchietto. Tuttavia, quando il vecchietto chiede direttamente a Pinocchio, che sta fingendo di non conoscersi, quest\'ultimo lo descrive come "un gran buon figliuolo, pieno di voglia di studiare, obbediente, affezionato al suo babbo e alla sua famiglia". Quindi sembra che ci siano descrizioni contrastanti sulla vera natura e personalità di Pinocchio.', source_nodes=[NodeWithScore(node=TextNode(id_='e84243e6-e394-435c-aa25-6e75d248cb75', embedding=None, metadata={'file_path': '/Users/dggallit/Documents/GenAI/personal-projects-and-experiments/notebooks/llama-index-learning/data/pinocchio.txt', 'file_name': 'pinocchio.txt', 'file_type': 'text/plain', 'file_size': 281679, 'creation_date': '2024-02-15', 'last_modified_date': '2024-02-15'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_s

# Prod

In [40]:
%pip install llama-index-vector-stores-opensearch llama-index-readers-elasticsearch -qU
%pip install llama-index-storage-docstore-dynamodb llama-index-storage-index-store-dynamodb-store llama-index-storage-kvstore-dynamodb -qU

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import boto3
# Create the DDB Tables


In [ ]:
from llama_index.storage.kvstore.dynamodb import DynamoDBKVStore
from llama_index.storage.docstore.dynamodb import DynamoDBDocumentStore
from llama_index.storage.index_store.dynamodb import DynamoDBIndexStore
from llama_index.vector_stores.opensearch import OpensearchVectorStore, OpensearchVectorClient

from llama_index.core import StorageContext

storage_context = StorageContext.from_defaults(
    docstore=DynamoDBDocumentStore(DynamoDBKVStore.from_table_name("books-docstore-table")), 
    index_store=DynamoDBIndexStore(DynamoDBKVStore.from_table_name("books-index-table")), 
    vector_store=OpensearchVectorStore(OpensearchVectorClient(
        endpoint="XXXXXXXXXXXXXXXXXXXXX", index="books-vector-store",
        embedding_field="embedding", text_field="text",
    )),
)